# Fedbiomed Researcher base example

## Introduction

This tutorial shows how to do 2d image classification example on MedNIST dataset using pretrained PyTorch model Densnet121.


## Load MedNIST to the node
### About MedNIST

MedNIST provides an artificial 2d classification dataset created by gathering different medical imaging datasets from TCIA, the RSNA Bone Age Challenge, and the NIH Chest X-ray dataset. The dataset is kindly made available by Dr. Bradley J. Erickson M.D., Ph.D. (Department of Radiology, Mayo Clinic) under the Creative Commons CC BY-SA 4.0 license.

MedNIST dataset is downloaded from the resources provided by the project MONAI:
https://github.com/Project-MONAI/MONAI-extra-test-data/releases/download/0.8.1/MedNIST.tar.gz

The dataset MedNIST has 58954 images of size (3, 64, 64) distributed into 6 classes (10000 images per class except for BreastMRI class which has 8954 images). It has the structure:


└── MedNIST/

    ├── AbdomenCT/
    
    └── BreastMRI/
    
    └── CXR/
    
    └── ChestCT/
    
    └── Hand/
    
    └── HeadCT/   


### Start the network

Before running this notebook, start the network with `./scripts/fedbiomed_run network`

### Setup the node

1. Populate the node with MedNIST dataset`./scripts/fedbiomed_run node add`
  * Select option 3 (mednist) to add MedNIST to the node
  * Confirm mednist tags tags ['#MEDNIST', '#dataset'] by hitting "y" and ENTER
  * Select the folder where MedNIST is downloaded (It will be downloaded if it is not found in the selected path)
  * Data must have been added (if you get a warning saying that data must be unique is because it's been already added)
  
2. Check that your data has been added by executing `./scripts/fedbiomed_run node list`
3. Start the node using `./scripts/fedbiomed_run node run`. Wait until you get `Starting task manager`. it means you are online.



## Start Fed-Biomed Researcher

We are now ready to start the researcher environment with the command source ./scripts/fedbiomed_environment researcher, and open the Jupyter notebook.

To make sure that MedNIST dataset is loaded in the node we can send a request to the network to list the available dataset in the node. The list command should output an entry for mednist data.

In [1]:
from fedbiomed.researcher.requests import Requests
req = Requests()
req.list(verbose=True)

2022-04-14 13:39:26,035 fedbiomed INFO - Component environment:
2022-04-14 13:39:26,036 fedbiomed INFO - type = ComponentType.RESEARCHER
2022-04-14 13:39:26,249 fedbiomed INFO - Messaging researcher_52f9c0d9-e21c-4653-9938-7a88e3e83cc0 successfully connected to the message broker, object = <fedbiomed.common.messaging.Messaging object at 0x7f4695883670>
2022-04-14 13:39:26,271 fedbiomed INFO - Listing available datasets in all nodes... 
2022-04-14 13:39:36,285 fedbiomed INFO - 
 Node: node_f6b39db8-f72c-4a0e-86bc-1cb1feedd85b | Number of Datasets: 2 
+---------+-------------+--------------------------+-----------------+--------------------+
| name    | data_type   | tags                     | description     | shape              |
+=========+=============+==========================+=================+====================+
| MNIST   | default     | ['#MNIST', '#dataset']   | MNIST database  | [60000, 1, 28, 28] |
+---------+-------------+--------------------------+-----------------+------

{'node_f6b39db8-f72c-4a0e-86bc-1cb1feedd85b': [{'name': 'MNIST',
   'data_type': 'default',
   'tags': ['#MNIST', '#dataset'],
   'description': 'MNIST database',
   'shape': [60000, 1, 28, 28]},
  {'name': 'MEDNIST',
   'data_type': 'mednist',
   'tags': ['#MEDNIST', '#dataset'],
   'description': 'MEDNIST dataset',
   'shape': [58954, 3, 64, 64]}]}

Use for developing (autoreloads changes made across packages)

In [2]:
%load_ext autoreload
%autoreload 2

## Define an experiment to train a model on the data

Declare a torch.nn MyTrainingPlan class to send for training on the node

In [3]:
import torch
import torch.nn as nn
from fedbiomed.common.training_plans import TorchTrainingPlan
from fedbiomed.common.data import DataManager
from torchvision import datasets, transforms
from torchvision.models import densenet121

# Here we define the model to be used. 
# You can use any class name (here 'Net')
class MyTrainingPlan(TorchTrainingPlan):
    def __init__(self, model_args: dict = {}):
        super(MyTrainingPlan, self).__init__(model_args)
        
        # Here we define the custom dependencies that will be needed by our custom Dataloader
        # In this case, we need the torch DataLoader classes
        # Since we will train on MNIST, we need datasets and transform from torchvision
        deps = ["from torchvision import datasets, transforms",
                "from torchvision.models import densenet121"]
        
        self.add_dependency(deps)
        
        self.model =  densenet121(pretrained=True)
        # We re-define the final fully-connected the layer
        self.model.classifier =nn.Sequential(nn.Linear(1024,512), nn.ReLU())
        self.loss_function = torch.nn.CrossEntropyLoss()

        
    def forward(self, x):
        return self.model(x)

    def training_data(self, batch_size = 48):
        # Custom torch Dataloader for MedNIST data
        preprocess = transforms.Compose([transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
        train_data = datasets.ImageFolder(self.dataset_path,transform = preprocess)
        train_kwargs = {'batch_size': batch_size, 'shuffle': True}
        return DataManager(dataset=train_data, **train_kwargs)
    
    def training_step(self, data, target):
        output = self.forward(data)
        loss   = self.loss_function(output, target)
        return loss


This group of arguments correspond respectively:
* `model_args`: a dictionary with the arguments related to the model (e.g. number of layers, features, etc.). This will be passed to the model class on the node side.
* `training_args`: a dictionary containing the arguments for the training routine (e.g. batch size, learning rate, epochs, etc.). This will be passed to the routine on the node side.

**NOTE:** typos and/or lack of positional (required) arguments will raise error. 🤓

In [4]:
model_args = {}

training_args = {
    'batch_size': 48, 
    'lr': 1e-3, 
    'epochs': 1, 
    'dry_run': False,  
    'batch_maxnum': 100 # Fast pass for development : only use ( batch_maxnum * batch_size ) samples
}

## Declare and run the experiment

- search nodes serving data for these `tags`, optionally filter on a list of node ID with `nodes`
- run a round of local training on nodes with model defined in `model_path` + federation with `aggregator`
- run for `round_limit` rounds, applying the `node_selection_strategy` between the rounds

In [6]:
from fedbiomed.researcher.experiment import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage

tags =  ['#MEDNIST', '#dataset']
rounds = 1

exp = Experiment(tags=tags,
                 model_args=model_args,
                 model_class=MyTrainingPlan,
                 training_args=training_args,
                 round_limit=rounds,
                 aggregator=FedAverage(),
                 node_selection_strategy=None)

2022-04-14 13:40:10,220 fedbiomed INFO - Searching dataset with data tags: ['#MEDNIST', '#dataset'] for all nodes
2022-04-14 13:40:20,233 fedbiomed INFO - Node selected for training -> node_f6b39db8-f72c-4a0e-86bc-1cb1feedd85b
2022-04-14 13:40:20,402 fedbiomed DEBUG - Model file has been saved: /home/gentoo/Projects/Fedbiomed/fedbiomed/var/experiments/Experiment_0004/my_model_5ad29141-25f4-41d5-ad1f-e8641331b31e.py
2022-04-14 13:40:20,502 fedbiomed DEBUG - upload (HTTP POST request) of file /home/gentoo/Projects/Fedbiomed/fedbiomed/var/experiments/Experiment_0004/my_model_5ad29141-25f4-41d5-ad1f-e8641331b31e.py successful, with status code 201
2022-04-14 13:40:21,211 fedbiomed DEBUG - upload (HTTP POST request) of file /home/gentoo/Projects/Fedbiomed/fedbiomed/var/experiments/Experiment_0004/aggregated_params_init_e9b1f19b-8c53-4f8b-bff2-b53e39c5ea49.pt successful, with status code 201


Let's start the experiment.

By default, this function doesn't stop until all the `round_limit` rounds are done for all the nodes

In [ ]:
exp.run(rounds=rounds, increase=True)

2022-04-14 13:40:31,346 fedbiomed INFO - Sampled nodes in round 0 ['node_f6b39db8-f72c-4a0e-86bc-1cb1feedd85b']
2022-04-14 13:40:31,347 fedbiomed INFO - Sending request 
					 To: node_f6b39db8-f72c-4a0e-86bc-1cb1feedd85b 
					 Request: : Perform training with the arguments: {'researcher_id': 'researcher_52f9c0d9-e21c-4653-9938-7a88e3e83cc0', 'job_id': '9c086d24-c5e5-4bdd-9c79-18b53dfdea75', 'training_args': {'test_ratio': 0.0, 'test_on_local_updates': False, 'test_on_global_updates': False, 'test_metric': None, 'test_metric_args': {}, 'batch_size': 48, 'lr': 0.001, 'epochs': 1, 'dry_run': False, 'batch_maxnum': 100}, 'training': True, 'model_args': {}, 'command': 'train', 'model_url': 'http://localhost:8844/media/uploads/2022/04/14/my_model_5ad29141-25f4-41d5-ad1f-e8641331b31e.py', 'params_url': 'http://localhost:8844/media/uploads/2022/04/14/aggregated_params_init_e9b1f19b-8c53-4f8b-bff2-b53e39c5ea49.pt', 'model_class': 'MyTrainingPlan', 'training_data': {'node_f6b39db8-f72c-4a0e-86

Local training results for each round and each node are available via `exp.training_replies()` (index 0 to (`rounds` - 1) ).

For example you can view the training results for the last round below.

Different timings (in seconds) are reported for each dataset of a node participating in a round :
- `rtime_training` real time (clock time) spent in the training function on the node
- `ptime_training` process time (user and system CPU) spent in the training function on the node
- `rtime_total` real time (clock time) spent in the researcher between sending the request and handling the response, at the `Job()` layer

In [ ]:
print("\nList the training rounds : ", exp.training_replies().keys())

print("\nList the nodes for the last training round and their timings : ")
round_data = exp.training_replies()[rounds - 1].data()
for c in range(len(round_data)):
    print("\t- {id} :\
    \n\t\trtime_training={rtraining:.2f} seconds\
    \n\t\tptime_training={ptraining:.2f} seconds\
    \n\t\trtime_total={rtotal:.2f} seconds".format(id = round_data[c]['node_id'],
        rtraining = round_data[c]['timing']['rtime_training'],
        ptraining = round_data[c]['timing']['ptime_training'],
        rtotal = round_data[c]['timing']['rtime_total']))
print('\n')
    
exp.training_replies()[rounds - 1].dataframe()

Federated parameters for each round are available via `exp.aggregated_params()` (index 0 to (`rounds` - 1) ).

For example you can view the federated parameters for the last round of the experiment :

In [ ]:
print("\nList the training rounds : ", exp.aggregated_params().keys())

print("\nAccess the federated params for the last training round :")
print("\t- params_path: ", exp.aggregated_params()[rounds - 1]['params_path'])
print("\t- parameter data: ", exp.aggregated_params()[rounds - 1]['params'].keys())


Feel free to run other sample notebooks or try your own models :D